In [ ]:
!pip install wikipedia==1.4.0 youtube-search==2.1.2

In [ ]:
#6.2 Building Agents using LangChain

from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.tools import WriteFileTool
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import YouTubeSearchTool
import os

api_key = ''
os.environ['OPENAI_API_KEY'] = api_key

youtube =  YouTubeSearchTool()
wiki  = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = [
    Tool(
        name="write",
        func=youtube.run,
        description="helps in getting youtube videos",
    ),
    Tool(
        name="wiki",
        func=wiki.run,
        description="Useful to search about a popular entity",
    )
]

prefix = """Answer the questions asked. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)
memory = ConversationBufferMemory(memory_key="chat_history")

llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)
agent_chain.run(input="Tell me something about SRK. Also, get me the link to one of his movies from youtube")



> Entering new AgentExecutor chain...
Thought: I should use the write and wiki tools to answer this question
Action: wiki
Action Input: SRK

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Shah Rukh Khan
Summary: Shah Rukh Khan (pronounced [ˈʃɑːɦɾʊx xɑːn]; born 2 November 1965), also known by the initialism SRK, is an Indian actor and film producer who works in Hindi films. Referred to in the media as the "Baadshah of Bollywood" and "King Khan", he has appeared in more than 90 films, and earned numerous accolades, including 14 Filmfare Awards. He has been awarded the Padma Shri by the Government of India, as well as the Ordre des Arts et des Lettres and Legion of Honour by the Government of France. Khan has a significant following in Asia and the Indian diaspora worldwide. In terms of audience size and income, several media outlets have described him as one of the most successful film stars in the world. Many of his films thematise Indian national identity and connections with diaspora communities, or gender, racial, social and religious differences and grievances.
Khan began his career with appearances in several television series in the late 1980s a

'Shah Rukh Khan is an Indian actor, producer and television personality who works in Hindi films. He has appeared in more than 90 films, and earned numerous accolades, including 14 Filmfare Awards. The link to one of his movies from youtube is https://www.youtube.com/watch?v=k8YiqM0Y-78&pp=ygUJU1JLIG1vdmll.'

In [ ]:
import langchain.tools as tools
tools.__all__

['AINAppOps',
 'AINOwnerOps',
 'AINRuleOps',
 'AINTransfer',
 'AINValueOps',
 'AIPluginTool',
 'APIOperation',
 'ArxivQueryRun',
 'AzureCogsFormRecognizerTool',
 'AzureCogsImageAnalysisTool',
 'AzureCogsSpeech2TextTool',
 'AzureCogsText2SpeechTool',
 'AzureCogsTextAnalyticsHealthTool',
 'BaseGraphQLTool',
 'BaseRequestsTool',
 'BaseSQLDatabaseTool',
 'BaseSparkSQLTool',
 'BaseTool',
 'BearlyInterpreterTool',
 'BingSearchResults',
 'BingSearchRun',
 'BraveSearch',
 'ClickTool',
 'CopyFileTool',
 'CurrentWebPageTool',
 'DeleteFileTool',
 'DuckDuckGoSearchResults',
 'DuckDuckGoSearchRun',
 'E2BDataAnalysisTool',
 'EdenAiExplicitImageTool',
 'EdenAiObjectDetectionTool',
 'EdenAiParsingIDTool',
 'EdenAiParsingInvoiceTool',
 'EdenAiSpeechToTextTool',
 'EdenAiTextModerationTool',
 'EdenAiTextToSpeechTool',
 'EdenaiTool',
 'ElevenLabsText2SpeechTool',
 'ExtractHyperlinksTool',
 'ExtractTextTool',
 'FileSearchTool',
 'GetElementsTool',
 'GmailCreateDraft',
 'GmailGetMessage',
 'GmailGetThread',

In [ ]:
import langchain.agents.agent_toolkits
print(langchain.agents.agent_toolkits.__all__)

['AINetworkToolkit', 'AmadeusToolkit', 'AzureCognitiveServicesToolkit', 'FileManagementToolkit', 'GmailToolkit', 'JiraToolkit', 'JsonToolkit', 'MultionToolkit', 'NasaToolkit', 'NLAToolkit', 'O365Toolkit', 'OpenAPIToolkit', 'PlayWrightBrowserToolkit', 'PowerBIToolkit', 'SlackToolkit', 'SteamToolkit', 'SQLDatabaseToolkit', 'SparkSQLToolkit', 'VectorStoreInfo', 'VectorStoreRouterToolkit', 'VectorStoreToolkit', 'ZapierToolkit', 'create_json_agent', 'create_openapi_agent', 'create_pbi_agent', 'create_pbi_chat_agent', 'create_spark_sql_agent', 'create_sql_agent', 'create_vectorstore_agent', 'create_vectorstore_router_agent', 'create_conversational_retrieval_agent', 'create_retriever_tool']


In [ ]:
!pip install pyowm langchain-experimental==0.0.46 langchain==0.0.349

In [ ]:
#6.4 Custom Tools for Agents

from pyowm.owm import OWM
from pyowm.utils.config import get_default_config
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.utilities import SerpAPIWrapper
from langchain.agents.tools import Tool
from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

import os
os.environ['OPENAI_API_KEY'] = api_key

def get_weather(city):
    config_dict = get_default_config()
    config_dict['language'] = 'en'  # your language here, eg. French
    owm = OWM('95440732edc79dadaa4fd1907e2e14d8', config_dict)
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place('Hyderabad, India')

    temperature = str(observation.weather.temperature("celsius")["temp"])+'°C'
    humidity = str(observation.weather.humidity)+'%'
    wind = str(observation.weather.wind())+' m/s'
    status = observation.weather.detailed_status

    return {'temperature':temperature,'humidity':humidity,'wind':wind,'status':status}

class GetWeatherInput(BaseModel):
    """Inputs for get_current_stock_price"""

    city: str = Field(description="City name with country separated by comma")

class GetWeatherTool(BaseTool):
    name = "get_weather_details_of_a_city"
    description = """
        Useful to get weather details of a city.
        Mandatory input format is 'city, country'.
        """
    args_schema: Type[BaseModel] = GetWeatherInput

    def _run(self, city: str):
        weather = get_weather(city)
        return weather

    def _arun(self, city: str):
        raise NotImplementedError("this tool doesn't support async")

llm = ChatOpenAI(openai_api_key=api_key)

tools = [GetWeatherTool()]

model = ChatOpenAI(openai_api_key=api_key)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
agent.run('Should I visit Manali, India this december based on the weather conditions?')



> Entering new PlanAndExecute chain...
steps=[Step(value='Gather weather information for Manali, India in December.'), Step(value='Determine the typical weather conditions in Manali in December.'), Step(value='Analyze the weather conditions to assess the suitability for visiting.'), Step(value='Consider personal preferences and interests.'), Step(value="Make a decision whether to visit Manali in December based on the weather conditions and personal preferences.\n\nGiven the above steps taken, please respond to the user's original question.")]

> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "get_weather_details_of_a_city",
  "action_input": {
    "city": "Manali, India"
  }
}
```
Observation: {'temperature': '25.23°C', 'humidity': '44%', 'wind': "{'speed': 4.12, 'deg': 60} m/s", 'status': 'haze'}
Thought:The previous action was to get the weather details of Manali, India. The weather details obtained were a temperature of 25.23°C, humidity of 44%, wind speed of 4.

'Based on the weather conditions in Manali, India in December (temperature of 25.23°C, humidity of 44%, and haze as the weather status), it can be assessed that the weather is suitable for visiting. The wind speed of 4.12 m/s coming from the east-northeast direction (60 degrees) is also favorable. However, it is always recommended to check for real-time updates before planning any travel.'

In [ ]:
from pyowm.owm import OWM
from pyowm.utils.config import get_default_config
from typing import Type
from pydantic import BaseModel, Field
from langchain.tools import BaseTool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.utilities import SerpAPIWrapper
from langchain.agents.tools import tool

from langchain_experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner
import os

api_key = ''
os.environ['OPENAI_API_KEY'] = api_key

@tool
def get_weather(city):
    """Useful to get weather details of a city.Mandatory input format is 'city, country'"""
    config_dict = get_default_config()
    config_dict['language'] = 'en'  # your language here, eg. French
    owm = OWM('95440732edc79dadaa4fd1907e2e14d8', config_dict)
    mgr = owm.weather_manager()
    observation = mgr.weather_at_place('Hyderabad, India')

    temperature = str(observation.weather.temperature("celsius")["temp"])+'°C'
    humidity = str(observation.weather.humidity)+'%'
    wind = str(observation.weather.wind())+' m/s'
    status = observation.weather.detailed_status

    return {'temperature':temperature,'humidity':humidity,'wind':wind,'status':status}



llm = ChatOpenAI(openai_api_key=api_key)

tools = [get_weather]

model = ChatOpenAI(openai_api_key=api_key)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [ ]:
agent.run('Should I visit Manali, India this december based on the weather conditions?')